# 交替式随机基准测试

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

**交替式随机基准测试（Interleaved Randomized Benchmarking, IRB）**，是一项能够高效估计指定量子门的噪声影响的技术。该技术的方法以及基本原理与标准随机基准测试相似。该教程展示如何在[百度量子平台](https://quantum.baidu.com/)上使用 **IRB** 来刻画衡量量子门的性能。

## 理论
在介绍交替式随机基准测试之前，我们假定读者已经熟悉了有关 Clifford 门的内容。因为我们同样也需要使用 Clifford 门来搭建随机电路，但是该电路结构与标准随机基准测试（SRB）的电路有所不同。

### 随机电路结构

SRB 中常见的随机 Clifford 电路结构如下图所示

![random-clifford-circuit](figures/srb-clifford-circuit.png "图1：SRB 随机 Clifford 电路。")

图中我们先加入 $m$ 个随机的 Clifford 门，之后再加入一个逆门 $C_{m+1}$，表示为所有 $m$ 个 Clifford 门的逆作用，即：

$$
C_{m+1} := \left(C_1C_2\cdots C_{m-1}C_{m}\right)^{-1}.
$$

则该线路在理想情况下输出仍然为初始态：

$$
C_{m+1}C_{m}C_{m-1}\cdots C_2C_1\vert \phi \rangle = \vert \phi \rangle.
$$

若初始态为 $\vert 0 \rangle$，则在理想电路的情况下无论多少次制备测量，结果都为 $\vert 0 \rangle$。

而在 IRB 中，其随机电路结构如图所示

![random-clifford-circuit](figures/irb-clifford-circuit.png "图2：IRB 随机 Clifford 电路。")

图中的 $C_i, i=1,2,\cdots,m$ 是随机选取的 Clifford 门，而 $C$ 则是我们指定的量子门，需要注意的是该量子门只能为 Clifford 门。可以看到量子门交替作用在量子比特上，因此该方法称作交替式随机基准测试。最后同样加入一个逆门 $C_{m+1}$，表示为所有 $m$ 个 $CC_i$ Clifford 门的逆作用。

### 随机基准测试

可以看到 IRB 中所用到的随机基准测试的思想与 SRB 完全一致。其具体的流程如下：

1. 我们首先需要运行 SRB 得到 $C_i$ 平均门保真度（average gate fidelity, AGF）$f$。

2. 接着我们运行 IRB 得到的 $CC_i$ 平均门保真度 $f_c$。

3. 最后我们通过 $f_c$ 和 $f$ 来估计指定的量子门 $C$ 的平均错误率（average error rate）$r$。

我们估计该指定的量子门的平均错误率 $r$ 在 $\left[r^{est}-E,r^{est}+E\right]$ 的范围内[1]。其中 $r^{est}$ 和 $E$ 分别为，

$$
r^{est} = \frac{d-1}{d}\left(1-\frac{f_c}{f}\right).
$$

其边界 $E$ 为

$$
E = \min\left\{
        \frac{(d-1)\left[\vert f - f_\circ/f\vert + (1-f)\right]}{d},\;
        \frac{2(d^2-1)(1-f)}{fd^2} + \frac{4\sqrt{1-f}\sqrt{d^2-1}}{f}
\right\}.
$$


## 实践

### 单量子比特电路

单量子比特的电路如图所示

![single-qubit](figures/irb-single-clifford-circuit.png "图3：单量子比特电路。")

下面代码中我们将单独的量子门设定为 `H` 门，在此我们使用的具体的构造方法可查看 `Clifford` 类或是论文[2]。在每个深度下都构造 $20$ 个随机电路，测量并取平均值。运行代码，并查看运行展示的最终结果。

In [ ]:
import QCompute
from qcompute_qep.quantum.clifford import Clifford
from qcompute_qep.benchmarking.interleavedrb import InterleavedRB

# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
QCompute.Define.hubToken = "Token"
qc = QCompute.BackendName.CloudBaiduQPUQian

# 1 qubit IRB
single_qubits = [0]
# generate a specific Clifford H gate
pattern = [[['A2'],[],['C1'],[],['E1']]]
target_gate = Clifford(1, pattern)
irb = InterleavedRB()
single_qubit_results = irb.benchmark(qubits=single_qubits,
                                     target_gate = target_gate,
                                     qc=qc,
                                     repeats=20,
                                     shots=1024,
                                     seq_lengths=[1, 5, 10, 15, 20, 25])
irb.plot_results()

### 双量子比特电路
双量子比特电路如图所示

![double-qubit](figures/irb-two-clifford-circuit.png "图4：双量子比特电路。")

方法同上，我们执行代码查看结果

In [ ]:
# 2 qubit IRB
two_qubits = [0,1]
target_gate = Clifford(2)
irb2 = InterleavedRB()
two_qubits_results = irb2.benchmark(qubits=two_qubits,
                                   target_gate=target_gate,
                                   qc=qc,
                                   repeats=20,
                                   shots=1024,
                                   seq_lengths=[1, 2, 3, 4, 5, 6])
irb2.plot_results()

运行上述代码，我们即可得到深度 $m$ 下的测量结果和拟合曲线（蓝色和绿色），以及 $f_c$, $f$, $r$ 等参数。

我们可以通过下面的代码打印结果查看参数

In [ ]:
print('The fidelity of 1 qubit SRB is :',single_qubit_results['StandardRB']['f'])
print('The fidelity of 1 qubit IRB is :',single_qubit_results['InterleavedRB']['f'])
print('The error rate of target gate is :',single_qubit_results['target_gate']['r'])
print('The fidelity of 2 qubit SRB is :',two_qubits_results['StandardRB']['f'])
print('The fidelity of 2 qubit IRB is :',two_qubits_results['InterleavedRB']['f'])
print('The error rate of target gate is :',two_qubits_results['target_gate']['r'])

从单量子比特的结果，我们可以看到 IRB 所得出的指定的 `H` 门的保真度与在 SRB 使用方法进行估计是不同的，因为 SRB 方法得到的是平均的量子门保真度。至此我们就完成了 **IRB** 的基本流程，并计算得到了指定的 Clifford 门错误率，衡量了该量子门噪声的影响。用户可以通过定义不同的噪声，设置不同的参数来进行实验观察结果。想了解更多随机基准测试及其变种的可以参考这篇论文[3]。


## 参考资料

\[1\] Magesan, Easwar, et al. "Efficient measurement of quantum gate error by interleaved randomized benchmarking." [Physical Review Letters](https://arxiv.org/abs/1203.4550) 109.8 (2012): 080505.

\[2\] Selinger, Peter. "Generators and relations for n-qubit Clifford operators." [Logical Methods in Computer Science](https://arxiv.org/abs/1310.6813v4) 11 (2015).

\[3\] Helsen, Jonas, et al. "A general framework for randomized benchmarking." [arXiv](https://arxiv.org/abs/2010.07974) preprint arXiv:2010.07974 (2020).


